In [ ]:
# Function to Iterate throught the API and the fields
def save_results_to_csv(data):

    # Define columns mapping for each result type
    columns_mapping = {
        'organic_results': {
            'Position': 'position',
            'Title': 'title',
            'URL': 'url',
            'Domain': 'domain',
            'Displayed URL': 'displayed_url',
            'Snippet': 'snippet',
            'Cached Page URL': 'cached_page_url',
            'Related Pages URL': 'related_pages_url'
        },
        'ads': {
            'Position': 'position',
            'Block Position': 'block_position',
            'Title': 'title',
            'URL': 'url',
            'Tracking URL': 'tracking_url',
            'Displayed URL': 'displayed_url',
            'Description': 'description',
            'Sitelinks': 'sitelinks'
        },
        'request': { 
            'Success': 'success',
            'Processed Timestamp': 'processed_timestamp',
            'Search URL': 'search_url',
            'Total Time Taken': 'total_time_taken'
        },
        'search_parameters': { 
            'Engine': 'engine',
            'Query': 'query',
            'Type': 'type',
            'Device': 'device',
            'Google Domain': 'google_domain',
            'HL': 'hl',
            'GL': 'gl',
            'Page': 'page',
            'Num': 'num'
        },
        'search_information': { 
            'Total Results': 'total_results',
            'Time Taken Displayed': 'time_taken_displayed',
            'Did You Mean': 'did_you_mean',
            'Showing Result For': 'showing_results_for',
            'Query Displayed': 'query_displayed',
            'Detected Location': 'detected_location',
            'No Results for Original Query': 'no_results_for_original_query'
        }
    }

    # Create a timestamp
    timestamp = dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    current_date = dt.datetime.now()
    
    # Iterate over the queries
    for query_key in data:
        # print(f"Top-level key: {query_key}")

        # Iterate over the json result
        for json_result_type, json_inner_schema in data[query_key].items():
            # print(f"  Inner key: {json_result_type}")

            # Iterate throught the expected dictionary structure
            for given_result_type, given_inner_schema in columns_mapping.items():

                # Check if the json_result matches the exected result_type
                if json_result_type == given_result_type:
                    # print(f"    Inner key Match found! {json_result_type}")

                    # Create a CSV file path
                    csv_file_path = f'C:/Users/idbs9/Downloads/eczack_capstone_datalake/staging_zone/{json_result_type}/{current_date.year}/{current_date.month}/{current_date.day}/{json_result_type}_{timestamp}.csv'

                    # Create the necessary directories if they don't exist
                    os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

                    # Evaluate if the JSON inner value is a Dictionary
                    if isinstance(json_inner_schema, dict):

                        with open(csv_file_path, 'a', newline='', encoding='utf-8') as csv_file:
                            fieldnames = ['Query', 'Col', 'Value']
                            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                            # Write the header
                            csv_writer.writeheader()

                            # Iterate throught the JSON inners values
                            for json_inner_key, json_inner_value in json_inner_schema.items():

                                # Iterate throught the expected inner values
                                for given_inner_key, given_inner_value in given_inner_schema.items():

                                    # Evaluate if the JSON inner column matches the expected json values
                                    if json_inner_key == given_inner_value:

                                        # print(f"      Query: {query_key}, Col: {given_inner_key}, Val: {json_inner_value}")
                                        
                                        csv_writer.writerow({'Query': query_key, 'Col': given_inner_key, 'Value': json_inner_value})

                        print(f"{json_result_type.capitalize()} Dictionaries results for query '{query_key}' saved to {csv_file_path}")

                    # Evaluate if the JSON inner value is a List
                    elif isinstance(json_inner_schema, list):
                        # print(f"The row is a List! {json_inner_schema}")

                        with open(csv_file_path, 'a', newline='', encoding='utf-8') as csv_file:
                            csv_writer = csv.DictWriter(csv_file, fieldnames=['Query', 'Col', 'Value'])
                            csv_writer.writeheader()

                            for item in json_inner_schema:
                                # Iterate throught the expected inner values
                                matched_values = []

                                for given_inner_key, given_inner_value in given_inner_schema.items():

                                    # Check if the json_key is present in the current row
                                    if given_inner_value in item:
                                        
                                        matched_values.append({'Query': query_key, 'Col': given_inner_value, 'Value': item[given_inner_value]})
                                
                                for result in matched_values:
                                    csv_writer.writerow(result)


                            print(f"{json_result_type.capitalize()} Lists results for query '{query_key}' saved to {csv_file_path}")

                    else:
                        print(f"I don't know that it is! {json_inner_schema}")
